In [3]:
!pip install gym gym-retro

# Setup basic enviroment

In [4]:
import retro

In [5]:
#Creates the enviroment
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')

RuntimeError: Cannot create multiple emulator instances per process, make sure to call env.close() on each environment before creating a new one

In [2]:
#Close the enviroment
env.close()
del env

NameError: name 'env' is not defined

Sample game

In [2]:
import time
# Reset game to starting state
env = StreetFighterEnv(style='balanced')
obs = env.reset()

done = False
for game in range(1):
    while not done:
        if done:
            obs=env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        time.sleep(0.01)
        if reward != 0:
            print("Reward:", reward)

c:\Users\nicol\OneDrive\Documentos\Aprendizaje\StreetFighter2Reinforced\.conda\lib\site-packages\pyglet\libs\win32\__init__.py:318: UserWarning: Could not set COM MTA mode. Unexpected behavior may occur.
  warnings.warn("Could not set COM MTA mode. Unexpected behavior may occur.")


Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.2
Reward: -0.43886967061089455
Reward: -0.7354552767919444
Reward: -1.1830228471554152
Reward: -1.5801554393712693
Reward: -2.0813906484311646
Reward: -2.4787819918451977
Reward: -2.9356097652951805
Reward: -3.461797799699713
Reward: -3.865424623862049
Reward: -4.365583872575454
Reward: -4.785127662620977
Reward: -5.212166183683374
Reward: -5.592626462530294
Reward: -6.006561154726521
Reward: -6.391766225761029
Reward: -6.724233913695421
Reward: -7.084

## Setup custom enviroment

In [ ]:
!pip install matplotlib

In [ ]:
from gym import Env
from gym.spaces import MultiBinary, Box
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118

  You can safely remove it manually.



     ---------------------------------------- 0.0/4.9 MB ? eta -:--:--
     - -------------------------------------- 0.2/4.9 MB 3.9 MB/s eta 0:00:02
     ----- ---------------------------------- 0.7/4.9 MB 7.4 MB/s eta 0:00:01
     -------------- ------------------------- 1.8/4.9 MB 12.8 MB/s eta 0:00:01
     ---------------------------------- ----- 4.3/4.9 MB 22.9 MB/s eta 0:00:01
     ---------------------------------------- 4.9/4.9 MB 24.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
     ------------------------------------ --- 3.6/4.0 MB 76.0 MB/s eta 0:00:01
     ---------------------------------------- 4.0/4.0 MB 62.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.7 GB 1.8 MB/s eta 0:25:01
     ---------------------------------------- 0.0/2.7 GB 2.3 MB/s eta 0:19:16
     ---

In [1]:
import retro
import numpy as np
import cv2
from gym import Env
from gym.spaces import Box, MultiBinary
import math

class StreetFighterEnv(Env):
    def __init__(self, style='balanced'):
        super().__init__()
        self.style = style
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED)
        self.previous_frame = None
        self.score = 0
        self.previous_health = None  # Valor inicial de salud
        self.opponent_previous_health = None
        self.last_hit_time = None

    def reset(self):
        obs = self.game.reset()
        obs = self.preprocess(obs)
        self.previous_frame = obs
        self.score = 0
        self.previous_health = 176
        self.opponent_previous_health = 176
        self.last_hit_time = 39208  # Inicializamos al valor máximo del round timer
        return obs

    def preprocess(self, observation):
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_CUBIC)
        channels = np.reshape(resize, (84, 84, 1))
        return channels

    def step(self, action):
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs

        current_health = info.get('health', self.previous_health)
        opponent_current_health = info.get('enemy_health', self.opponent_previous_health)

        # Penalizaciones y recompensas según el estilo
        distance_x = info['enemy_x_position'] - info['x_position']
        distance_y = info['enemy_y_position'] - info['y_position']
        distance = math.sqrt(distance_x**2 + distance_y**2)
        round_timer = info['round_timer']

        # Verificamos si se hizo daño al oponente
        if opponent_current_health < self.opponent_previous_health:
            self.last_hit_time = round_timer

        time_since_last_hit = (39208 - self.last_hit_time) / 1000  # Convertimos a segundos

        if self.style == 'defensive':
            # Penalización por baja salud
            health_penalty = -0.5 * (176 - current_health)
            reward += health_penalty
            
        elif self.style == 'balanced':
            # Recompensa por mantener una distancia óptima del enemigo
            optimal_distance = 100  # Supongamos que la distancia óptima es 100
            distance_reward = -0.1 * abs(distance - optimal_distance)
            reward += distance_reward
            
            # Penalización por tiempo sin pegar un golpe que hizo daño
            time_penalty = -0.005 * time_since_last_hit
            reward += time_penalty
            
        elif self.style == 'aggressive':
            # Penalización por tiempo sin atacar
            time_penalty = -0.05 * time_since_last_hit
            reward += time_penalty
            
            # Recompensa por estar cerca del enemigo
            distance_reward = -0.1 * distance
            reward += distance_reward
            
            # Recompensa por reducir la salud del enemigo
            enemy_health_reward = 0.5 * (176 - opponent_current_health)
            reward += enemy_health_reward
            
            # Penalización por baja salud
            health_penalty = -0.2 * (176 - current_health)
            reward += health_penalty

        self.previous_health = current_health
        self.opponent_previous_health = opponent_current_health
        return frame_delta, reward, done, info

    def render(self, *args, **kwargs):
        self.game.render()

    def close(self):
        self.game.close()




In [1]:
import retro
import numpy as np
import cv2
from gym import Env
from gym.spaces import Box, MultiBinary
# Create custom environment 
class StreetFighter(Env): 
    def __init__(self):
        super().__init__()
        # Specify action space and observation space 
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        # Startup and instance of the game 
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED)
    
    def reset(self):
        # Return the first frame 
        obs = self.game.reset()
        obs = self.preprocess(obs) 
        self.previous_frame = obs 
        
        # Create a attribute to hold the score delta 
        self.score = 0 
        return obs
    
    def preprocess(self, observation): 
        # Grayscaling 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Resize 
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        # Add the channels value
        channels = np.reshape(resize, (84,84,1))
        return channels 
    
    def step(self, action): 
        # Take a step 
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs) 
        
        # Frame delta 
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs 
        
        # Reshape the reward function
        #reward = info['score'] - self.score 
        self.score = info['score'] 
        if reward != 0:
            print("reward", reward)
        
        return frame_delta, reward, done, info
    
    def render(self, *args, **kwargs):
        self.game.render()
        
    def close(self):
        self.game.close()

In [2]:
try:
    env.close()
except:
    print("Environment closed")

Environment closed


In [2]:
import optuna 
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import os 

c:\Users\nicol\OneDrive\Documentos\Aprendizaje\StreetFighter2Reinforced\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
LOG_DIR = './logs/'
OPT_DIR = './opt/'

In [4]:
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [5]:
SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(1))

In [6]:

def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial) 

        # Create environment 
        env = StreetFighterEnv(style="balanced")
        env = Monitor(env, LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')

        # Create algo 
        model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
        #model.learn(total_timesteps=300)
        model.learn(total_timesteps=100000)

        # Evaluate model 
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()

        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward

    except Exception as e:
        return -1000

In [9]:
study = optuna.create_study(direction='maximize', study_name='mi_estudio')


[I 2024-06-22 00:45:58,981] A new study created in memory with name: mi_estudio


In [ ]:
study = optuna.create_study(direction='maximize', study_name='mi_estudio')
study.optimize(optimize_agent, n_trials=100, n_jobs=1)

In [19]:
optuna.study.load_study(study_name='mi_estudio')

TypeError: load_study() missing 1 required keyword-only argument: 'storage'

In [10]:
study.best_params 

ValueError: No trials are completed yet.

In [11]:
model = PPO.load(os.path.join(OPT_DIR, 'trial_0_best_model.zip'))

In [12]:
from stable_baselines3.common.callbacks import BaseCallback

In [13]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [14]:
CHECKPOINT_DIR = './train/'

In [15]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [16]:
# Create environment 
env = StreetFighterEnv(style="balanced")
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [18]:
model_params = model.best_params
model_params['n_steps'] = 7488  # set n_steps to 7488 or a factor of 64
# model_params['learning_rate'] = 5e-7
model_params

AttributeError: 'PPO' object has no attribute 'best_params'

In [16]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)

NameError: name 'model_params' is not defined

In [18]:
model.load(os.path.join(OPT_DIR, 'trial_6_best_model.zip'))

In [21]:
#model.learn(total_timesteps=1000, callback=callback)
model.learn(total_timesteps=1000000, callback=callback)

AttributeError: 'NoneType' object has no attribute 'reset'

In [17]:
model = PPO.load('./opt/trial_6_best_model.zip')
mean_reward, _ = evaluate_policy(model, env, render=True, n_eval_episodes=1)
mean_reward

c:\Users\nicol\OneDrive\Documentos\Aprendizaje\StreetFighter2Reinforced\.conda\lib\site-packages\pyglet\libs\win32\__init__.py:318: UserWarning: Could not set COM MTA mode. Unexpected behavior may occur.
  warnings.warn("Could not set COM MTA mode. Unexpected behavior may occur.")


-10773.421049

In [23]:
obs = env.reset()
obs.shape
env.step(model.predict(obs)[0])

(array([[[[  0,   0,  98,   0],
          [  0,   0, 103,   0],
          [  0,   0, 117,   0],
          ...,
          [  0,   0, 166,   0],
          [  0,   0, 166,   0],
          [  0,   0, 231,   0]],
 
         [[  0,   0,  76,   0],
          [  0,   0, 124,   0],
          [  0,   0, 144,   0],
          ...,
          [  0,   0, 166,   0],
          [  0,   0, 166,   0],
          [  0,   0, 231,   0]],
 
         [[  0,   0, 135,   0],
          [  0,   0, 150,   0],
          [  0,   0, 130,   0],
          ...,
          [  0,   0, 166,   0],
          [  0,   0, 159,   0],
          [  0,   0, 124,   0]],
 
         ...,
 
         [[  0,   0, 220,   0],
          [  0,   0, 192,   0],
          [  0,   0, 196,   0],
          ...,
          [  0,   0, 177,   0],
          [  0,   0, 193,   0],
          [  0,   0, 213,   0]],
 
         [[  0,   0, 162,   0],
          [  0,   0, 168,   0],
          [  0,   0, 238,   0],
          ...,
          [  0,   0, 196,   0],
 

In [18]:
import time
# Reset game to starting state
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)
        time.sleep(0.01)
        print(reward)

[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[0.]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.2]
[-0.3]
[-0.3]
[-0.3]
[-0.3]
[-0.3]
[-0.3]
[-0.3]
[-0.3]
[-0.3]
[-0.3]
[-0.3]
[-0.3]
[-0.3]
[-0.3]
[-0.3]
[-0.3]
[-0.4810715]
[-0.62290025]
[-0.7425706]
[-0.8321213]
[-0.8906641]
[-0.95234835]
[-0.9922256]
[-1.0100056]
[-1.0055573]
[-1.0306076]
[-1.0412054]
[-1.0372995]
[-1.0189087]
[-0.97281885]
[-0.99722266]
[-1.0127869]
[-1.0893322]
[-1.1608824]
[-1.2273151]
[-1.2216175]
[-1.3547701]
[-1.4853069]
[-1.6131054]
[-1.67